<a href="https://colab.research.google.com/github/atsiolis/predict_health_costs_with_regression/blob/main/Copy_of_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Preprocess categorical data
dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'])
dataset.tail()

In [ ]:
# Split data into training (80%) and testing (20%)
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

# Visualise the split
print(f"Train dataset shape: {train_dataset.shape}")
print(f"Test dataset shape: {test_dataset.shape}")

In [ ]:
# Separate labels (expenses column)
train_labels = train_dataset.pop("expenses")
test_labels = test_dataset.pop("expenses")


In [ ]:
# Normalize numerical data
def normalize_data(data, stats):
    # Select only numerical columns for normalization
    numerical_data = data.select_dtypes(include=[np.number])

    # Apply normalization only to numerical columns
    normalized_data = (numerical_data - stats['mean']) / stats['std']

    # Reattach the non-numerical columns (if any)
    non_numerical_data = data.select_dtypes(exclude=[np.number])

    # Combine normalized numerical data with the non-numerical data
    return pd.concat([normalized_data, non_numerical_data], axis=1)

train_stats = train_dataset.describe().transpose()
train_dataset = normalize_data(train_dataset, train_stats)
test_dataset = normalize_data(test_dataset, train_stats)

train_dataset

In [ ]:
# Handle NaN values
train_dataset = train_dataset.fillna(0)
test_dataset = test_dataset.fillna(0)
train_labels = train_labels.fillna(0)
test_labels = test_labels.fillna(0)

In [ ]:
# Build the model
def build_model():
    model = keras.Sequential([
        layers.Input(shape=[len(train_dataset.keys())]),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.5),
        layers.Dense(1)
    ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss='mse', metrics=['mae', 'mse'])
    return model

model = build_model()
model.summary()


In [ ]:
# Train the model
early_stop = keras.callbacks.EarlyStopping(monitor='val_mae', patience=10)
history = model.fit(
    train_dataset, train_labels,
    epochs=100, validation_split=0.2,
    verbose=1, callbacks=[early_stop]
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
